In [1]:
import os
import re
import datetime 
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq
import janitor
from random import sample

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
#establish Connection to GBQ

# These first two values will be different on your machine. 
service_path = "C:\\Users\\ofano\\Documents\\MSBA Folder\\MSBA Folder\\"
service_file = 'msba-project-2022-75bb8251ef6f.json' 
gbq_proj_id = 'msba-project-2022' 

# And this should stay the same. 
private_key =service_path + service_file

In [3]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [4]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [5]:
#let's make sure it works

for item in client.list_datasets() : 
    print(item.full_dataset_id)

msba-project-2022:dram_shop
msba-project-2022:wedge_dataset


# Query 1 Total Rows

In [6]:
query = """
        SELECT count(*) as cnt
        FROM `msba-project-2022.wedge_dataset.transArchive_*`
        """

total_rows = client.query(query).to_dataframe()
total_rows

,cnt
0,85760071


# Query 2 January/Oct 2012 Rows

In [7]:
query="""
    SELECT EXTRACT(YEAR from datetime) as Yr,
           EXTRACT(MONTH from datetime) as Mo,
           count(*)
    FROM `msba-project-2022.wedge_dataset.transArchive_*` 
    GROUP BY Yr, Mo
    HAVING Yr = 2012 
    ORDER BY Yr, Mo
    """


Jan_Oct_Rows = client.query(query).to_dataframe()
Jan_Oct_Rows

,Yr,Mo,f0_
0,2012,1,1070905
1,2012,2,1034261
2,2012,3,1130248
3,2012,4,1134998
4,2012,5,1132624
5,2012,6,1053912
6,2012,7,1042455
7,2012,8,1029592
8,2012,9,1044436
9,2012,10,1042285


# Query 3 Rows by Month

In [11]:
query="""
SELECT EXTRACT(Month from datetime) as Mo,
       count(*) as cnt
FROM `msba-project-2022.wedge_dataset.transArchive_*` 
--FROM `your_project.your_dataset.your_table_prefix_*` 
WHERE EXTRACT(Year from datetime) <= 2016
GROUP BY Mo
ORDER BY cnt
"""

Jan_Oct_Rows2 = client.query(query).to_dataframe()
Jan_Oct_Rows2
#-- January = ?
#-- October = ?


,Mo,cnt
0,2,6556768
1,12,6740728
2,11,6955361
3,9,6975852
4,8,7008746
5,6,7012154
6,7,7013003
7,1,7056752
8,10,7085742
9,4,7417377


# Query 4 Null Counts

In [12]:
query="""SELECT COUNTIF(trans_subtype is NULL) as Null_TS,
       COUNTIF(datetime is NULL) as Null_DT,
       COUNTIF(local IS NULL) as Null_Local,
       COUNTIF(card_no IS NULL) as Null_CN
FROM `msba-project-2022.wedge_dataset.transArchive_*`
        """

Null_Counts = client.query(query).to_dataframe()
Null_Counts


,Null_TS,Null_DT,Null_Local,Null_CN
0,0,0,0,0


# Query 5 High Volume Cards

In [13]:
query="""SELECT card_no, 
       COUNT(*) as cnt
FROM `msba-project-2022.wedge_dataset.transArchive_*`
WHERE card_no != 3
GROUP BY card_no
ORDER BY cnt desc
LIMIT 10
"""

high_vol_cards = client.query(query).to_dataframe()
high_vol_cards

##What card is #5 on the list and how many rows does it have?
## 10504



,card_no,cnt
0,12539,2161159
1,10499,1013696
2,14140,942225
3,20074,643454
4,14987,460625
5,10504,164864
6,21517,133006
7,23170,99276
8,19750,65875
9,15876,59052


# Query 6 Row Count 18736

In [14]:

query="""SELECT card_no, 
       COUNT(*) as cnt
FROM `msba-project-2022.wedge_dataset.transArchive_*`
WHERE card_no = 18736
GROUP BY card_no"""

row_count = client.query(query).to_dataframe()
row_count


##How many rows do I have?
#12153


,card_no,cnt
0,18736,12153


# Query 7 Popular Products

In [15]:
query="""
        SELECT LOWER(Description), 
               COUNT(*) as cnt
        FROM `msba-project-2022.wedge_dataset.transArchive_*` 
        WHERE trans_type = "I" AND
              Description != "Discount"
        GROUP BY LOWER(Description)
        ORDER BY cnt desc
        LIMIT 10
        """

popular_products = client.query(query).to_dataframe()
popular_products



## Which product is #1? How many rows does it have?
## Banana organic 908637
## Which product is #4? How many rows does it have?
##Broccoli organic 344657

,f0_,cnt
0,banana organic,908637
1,green patch redemption,572472
2,wedge cookie,510140
3,avocado hass organic,456771
4,broccoli organic,344657
5,celery organic,254479
6,citrus lemon organic,246058
7,salad mix organic,225088
8,pepper bell red organic,224162
9,spinach bulk organic,209826


# Query 8 Single Record Items

In [16]:
query = """
        SELECT 
	COUNT(DISTINCT Description) as SingleRecordItems
FROM (
	SELECT LOWER(Description) as Description,
		   COUNT(*) as cnt
	FROM `msba-project-2022.wedge_dataset.transArchive_*`
	--FROM `your_project.your_dataset.your_table_prefix_*` 
  WHERE trans_type = "I" AND Description != "Discount"
	GROUP BY Description
	HAVING cnt = 1
	)"""
        

single_record = client.query(query).to_dataframe()
single_record

,SingleRecordItems
0,2741


# Query 9 Owner Fractions by Year

In [17]:
query= """
SELECT
  Y,
  OwnerRows,
  NonOwnerRows,
  ROUND(OwnerRows/(OwnerRows+NonOwnerRows),4) AS OwnerFrac
FROM (
  SELECT
    EXTRACT(Year
    FROM
      datetime) AS Y,
    COUNTIF(card_no != 3) AS OwnerRows,
    COUNTIF(card_no = 3) AS NonOwnerRows
  FROM
    `msba-project-2022.wedge_dataset.transArchive_*`
    --`your_project.your_dataset.your_table_prefix_*` 
  GROUP BY
    Y)
ORDER BY
    Y
    """


owner_fraction = client.query(query).to_dataframe()
owner_fraction



## Which year has the highest portion of owner rows? What year? What fraction?
##

## Which year has the lowest portion of owner rows? What year? What fraction?
##

,Y,OwnerRows,NonOwnerRows,OwnerFrac
0,2010,9005823,3128481,0.7422
1,2011,8980791,3200836,0.7372
2,2012,9442976,3287078,0.7418
3,2013,9214653,3103827,0.7480
4,2014,9237990,2932258,0.7591
5,2015,9217689,3182911,0.7433
6,2016,8166860,2721159,0.7501
7,2017,703764,232975,0.7513
